In [3]:
import requests
import json

from functions import get_layer, to_geojson

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import os
import pandas as pd

length = 0

dir = os.listdir('./data/raw/xlsx/')
for i, file in enumerate(dir):
    df = pd.read_excel(f'./data/raw/xlsx/{file}')
    length += df.shape[0]
    print(f'[{i}/{len(dir)}] | {file} readed')


print(length)

[0/308] | Akaa.xlsx readed
[1/308] | Alajärvi.xlsx readed
[2/308] | Alavieska.xlsx readed
[3/308] | Alavus.xlsx readed
[4/308] | Asikkala.xlsx readed
[5/308] | Askola.xlsx readed
[6/308] | Aura.xlsx readed
[7/308] | Brändö.xlsx readed
[8/308] | Eckerö.xlsx readed
[9/308] | Enonkoski.xlsx readed
[10/308] | Enontekiö.xlsx readed
[11/308] | Espoo.xlsx readed
[12/308] | Eura.xlsx readed
[13/308] | Eurajoki.xlsx readed
[14/308] | Evijärvi.xlsx readed
[15/308] | Finström.xlsx readed
[16/308] | Forssa.xlsx readed
[17/308] | Föglö.xlsx readed
[18/308] | Geta.xlsx readed
[19/308] | Haapajärvi.xlsx readed
[20/308] | Haapavesi.xlsx readed
[21/308] | Hailuoto.xlsx readed
[22/308] | Halsua.xlsx readed
[23/308] | Hamina.xlsx readed
[24/308] | Hammarland.xlsx readed
[25/308] | Hankasalmi.xlsx readed
[26/308] | Hanko.xlsx readed
[27/308] | Harjavalta.xlsx readed
[28/308] | Hartola.xlsx readed
[29/308] | Hattula.xlsx readed
[30/308] | Hausjärvi.xlsx readed
[31/308] | Heinola.xlsx readed
[32/308] | Hein

In [6]:
import utm
import pandas as pd
import gspread
from geopy.geocoders import Nominatim


def get_address_slow(coordinates, address_format = 'fin', language = 'en'):
    '''proxy object: {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'}'''
    lat, lon = coordinates.split(' ')
    address_format = address_format.lower()

    location = None

    try:
        geolocator = Nominatim(user_agent="geoapiExercises", timeout=10, proxies={'https': '159.112.235.173:80'})
        location = geolocator.reverse((lat, lon), language=language, exactly_one=True)
    except Exception as ex:
        return 'error'

    if location:
        address = location.raw['address']
        full_address = location.raw['display_name']
        if address_format == 'fin':
            try:
                if address.get('house_number') and address.get('postcode'):
                    print('[i] converted address...')
                    return f"{address['road']} {address['house_number']}, {address['postcode']}, {address['city']}"
                elif address.get('postcode'):
                    print('[i] converted address...')
                    return f"{address['road']}, {address['postcode']}, {address['city']}"
                elif address.get('house_number'):
                    print('[i] converted address...')
                    return f"{address['road']} {address['house_number']}, {address['city']}"
                elif not (address.get('postcode') and address.get('house_number')):
                    print('[i] converted address...')
                    return f"{address['road']}, {address['city']}"
                else:
                    print(f'[i] converted address...')
                    return full_address
            except Exception as ex:
                # print(f'ERROR: {ex}\naddress: {address}\n')
                return full_address
        
        elif address_format == 'none':
            return address
    else:
        return 'error'

In [7]:
import pandas as pd

df = pd.read_excel('./data/raw/xlsx/Brändö.xlsx')
df.head()

,kayttotarkoitus,kerrosluku,ETRS-TM35FIN,"coordinates (lat, lon)"
0,6,1,171510.088 6738416.853,60.64614156880835 20.987469096198378
1,6,1,171521.927 6738405.806,60.646052633955314 20.987702835851135
2,6,1,171532.703 6738386.23,60.645886672135106 20.98793148094865
3,6,1,171488.609 6738396.595,60.645943079945056 20.98711231332102
4,6,1,171494.254 6738531.913,60.64715595768822 20.986989041652947


In [8]:
get_address_slow('60.64614156880835 20.987469096198378')

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=60.64614156880835&lon=20.987469096198378&format=json&accept-language=en&addressdetails=1 (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 400 Bad Request')))

In [4]:
df['address'] = df['coordinates (lat, lon)'].apply(get_address_slow)

KeyboardInterrupt: 

In [1]:
import multiprocessing
from functools import partial
import utm
import pandas as pd
import gspread
from geopy.geocoders import Nominatim
import os
import numpy as np


def get_address(coordinates, proxy, address_format = 'fin', language = 'en'):
    lat, lon = coordinates.split(' ')
    address_format = address_format.lower()

    location = None

    print(f'{coordinates}\n{proxy}===')


    # try:
    #     geolocator = Nominatim(user_agent="geoapiExercises", timeout=10, proxies=proxy)
    #     location = geolocator.reverse((lat, lon), language=language)
    # except Exception as ex:
    #     print(f'Error: {ex}')
    #     return 'error'

    # if location:
    #     address = location.raw['address']
    #     full_address = location.raw['display_name']
    #     if address_format == 'fin':
    #         try:
    #             if address.get('house_number') and address.get('postcode'):
    #                 print('[i] converted address...')
    #                 return f"{address['road']} {address['house_number']}, {address['postcode']}, {address['city']}"
    #             elif address.get('postcode'):
    #                 print('[i] converted address...')
    #                 return f"{address['road']}, {address['postcode']}, {address['city']}"
    #             elif address.get('house_number'):
    #                 print('[i] converted address...')
    #                 return f"{address['road']} {address['house_number']}, {address['city']}"
    #             elif not (address.get('postcode') and address.get('house_number')):
    #                 print('[i] converted address...')
    #                 return f"{address['road']}, {address['city']}"
    #             else:
    #                 print(f'[i] converted address...')
    #                 return full_address
    #         except Exception as ex:
    #             # print(f'ERROR: {ex}\naddress: {address}\n')
    #             return full_address
        
    #     elif address_format == 'none':
    #         return address
    # else:
    #     return 'error'


def worker(worker_id, work, proxy):
    current_works = work[worker_id]
    current_proxy = proxy[worker_id]
    for i, file in enumerate(current_works):
        filename = file.replace('.xlsx', '')
        df = pd.read_excel(f'./data/raw/xlsx/{file}')
        df['address'] = df['coordinates (lat, lon)'].apply(lambda coordinates: get_address(coordinates, proxy=current_proxy))
        # Переименовать столбцы 1,2
        # df.to_excel(f'./data/output/{filename}.xlsx')
        # print(f'[{i}/{len(current_works)}] @ File {filename} converted and saved')
    

if __name__ == '__main__':
    proxy = [
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
        {'http': '213.209.133.65:63280', 'login': 'ucgXeK84', 'password': '4FGTCYH8'},
    ]

    directory_output = [file.replace('.xlsx', '') for file in os.listdir('./data/raw/xlsx/')]
    directory = [file for file in os.listdir('./data/raw/xlsx/') if file.replace('.xlsx', '') not in directory_output]

    workers_count = len(proxy)
    work = np.array_split(directory, workers_count)

    with multiprocessing.Pool(processes=workers_count) as pool:
        worker_func = partial(get_address, work=work, proxy=proxy)
        pool.map(worker_func, range(workers_count))
    print('Finished!')

In [7]:
import pandas as pd

proxy = pd.read_csv('./data/hideme_proxy_export.csv', sep=';')

In [17]:
for index, row in proxy.iterrows():
    print(row['ip'])

172.67.176.13
203.23.106.194
172.67.191.247
172.67.68.216
203.13.32.22
185.162.228.144
185.162.231.66
203.28.8.216
203.32.120.184
173.245.49.111
45.14.174.142
203.23.106.95
66.235.200.110
172.67.182.23
203.32.120.207
203.24.108.178
203.28.9.227
203.30.191.205
45.8.105.169
185.162.229.181
45.14.174.56
185.162.231.134
203.23.106.22
31.43.179.36
172.67.117.192
172.67.212.220
23.227.38.4
203.13.32.26
185.162.230.53
203.23.106.232
203.32.121.98
203.24.108.62
203.22.223.248
45.8.107.78
203.24.103.12
45.12.31.204
203.32.120.133
50.170.90.28
23.227.38.147
185.162.231.180
31.43.179.204
172.67.185.193
66.235.200.93
172.67.167.16
172.67.167.23
172.67.167.75
45.131.5.6
172.67.88.82
172.67.85.150
203.24.102.176
203.24.109.227
203.24.108.2
45.14.174.208
23.227.38.223
203.24.102.143
203.32.121.228
203.13.32.8
185.162.231.177
203.30.191.129
45.12.30.25
31.43.179.165
172.67.133.8
188.114.99.206
66.235.200.194
172.67.146.112
45.8.106.17
203.30.189.89
185.162.231.234
203.28.9.96
203.30.188.187
23.227.38.

In [10]:
from geopy.geocoders import Nominatim

In [11]:
cord = '60.20698104241428 24.99696950947328'.replace(' ', ', ')

In [12]:
result = []

In [13]:
geolocator = Nominatim(user_agent='geoapiExercises', timeout=15)
location = geolocator.reverse(cord, language='en')
address = location.raw['address']
if location:
    address = location.raw['address']
    full_address = location.raw.get('display_name')
    print(full_address)
    try:
        if address.get('house_number') and address.get('postcode'):
            result.append(f"{address['road']} {address['house_number']}, {address['postcode']}, {address['city']}")
        elif address.get('postcode'):
            result.append(f"{address['road']}, {address['postcode']}, {address['city']}")
        elif address.get('house_number'):
            result.append(f"{address['road']} {address['house_number']}, {address['city']}")
        elif not (address.get('postcode') and address.get('house_number')):
            result.append(f"{address['road']}, {address['city']}")
        else:
            result.append(full_address)
    except:
        result.append('error')
else:
    result.append('error')

40, Pornaistenniemen luontopolku, Viikinranta, Viikki, Northeastern major district, Helsinki, Helsinki sub-region, Uusimaa, Southern Finland, Mainland Finland, 00710, Finland


In [14]:
result

['Pornaistenniemen luontopolku 40, 00710, Helsinki']

In [3]:
import os

In [4]:
dir = os.listdir('./data/geojson/')

In [5]:
done = ''

In [6]:
for file in dir:
    done += f"{file}\n"

In [7]:
print(done)

Akaa.geojson
Alajärvi.geojson
Alavieska.geojson
Alavus.geojson
Asikkala.geojson
Askola.geojson
Aura.geojson
Brändö.geojson
Eckerö.geojson
Enonkoski.geojson
Enontekiö.geojson
Espoo.geojson
Eura.geojson
Eurajoki.geojson
Evijärvi.geojson
Finström.geojson
Forssa.geojson
Föglö.geojson
Geta.geojson
Haapajärvi.geojson
Haapavesi.geojson
Hailuoto.geojson
Halsua.geojson
Hamina.geojson
Hammarland.geojson
Hankasalmi.geojson
Hanko.geojson
Harjavalta.geojson
Hartola.geojson
Hattula.geojson
Hausjärvi.geojson
Heinola.geojson
Heinävesi.geojson
Helsinki.geojson
Hirvensalmi.geojson
Hollola.geojson
Huittinen.geojson
Humppila.geojson
Hyrynsalmi.geojson
Hyvinkää.geojson
Hämeenkyrö.geojson
Hämeenlinna.geojson
Ii.geojson
Iitti.geojson
Ikaalinen.geojson
Ilmajoki.geojson
Ilomantsi.geojson
Imatra.geojson
Inari.geojson
Ingå.geojson
Isojoki.geojson
Isokyrö.geojson
Jakobstad.geojson
Janakkala.geojson
Joensuu.geojson
Jokioinen.geojson
Jomala.geojson
Joroinen.geojson
Joutsa.geojson
Juuka.geojson
Juupajoki.geojson
Juv

In [9]:
import requests

In [26]:
response = requests.get(f'https://photon.komoot.io/reverse?lon=&lat=')
response.text

'{"features":[{"geometry":{"coordinates":[73.24246253951605,55.02698115],"type":"Point"},"type":"Feature","properties":{"osm_id":152210717,"extent":[73.2415584,55.0271719,73.2433646,55.0267699],"country":"Россия","city":"Омск","countrycode":"RU","postcode":"644000","type":"locality","osm_type":"W","osm_key":"landuse","district":"Юбилейный","osm_value":"retail","name":"Строительный рынок","state":"Омская область"}}],"type":"FeatureCollection"}'

In [12]:
result

NameError: name 'result' is not defined